# Step 3: Testing
This file is used for testing the trained classification model. In this case, the model is CNN (InceptionResNetV2).

<br>

**Requirments**
- Perform preprocessing in Step1_Preprocessing.ipynb.
- Manually create the folders (i.e., , InceptionResNetV2/modelWeight, and InceptionResNetV2/pic).
- Download the model file (InceptionResNetV2.h5) to InceptionResNetV2/model or run Step2_Training.ipynb.
- Define the data paths in the Setup section.

## Setup
Version of libraries
- Tensorflow: 2.4.0
- Keras: 2.4.0
- pandas: 1.2.1
- matplotlib 3.3.3
- numpy 1.19.5
- scikit-learn 0.24.1

In [ ]:
# Import the library for data importing and testing process
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import keras
from keras.models import load_model
from keras import backend as K
from keras.applications import InceptionResNetV2
from sklearn.metrics import confusion_matrix, classification_report
import itertools

In [ ]:
# Define paths for testing dataset and the trained model
testDir = '/xxx/data/Test'
model_file = "/xxx/InceptionResNetV2/model/InceptionResNetV2.h5"

## Testing process
Use the trained model to predict the organ classes.

In [ ]:
# Load model from the pre-defined path.
test_model = load_model(model_file)
print('load model')

In [ ]:
# Import data from Testing dataset directory with ImageDataGenerator from Keras
imgDataGen = ImageDataGenerator()

# Target size is images size for testing which we use (256,256)
test_batches = imgDataGen.flow_from_directory( testDir, target_size = (256, 256), color_mode='rgb', batch_size = 10, class_mode ='categorical', shuffle=False)

In [ ]:
# Predict the testing dataset using loaded model
# steps define how many batches of samples to use in one epoch for testing dataset when predict. But we use 1 batch_size so the batches of samples is equal to number of images.
predictions_test = test_model.predict(test_batches,steps = test_batches.n // test_batches.batch_size, verbose = 1)

In [ ]:
# After prediction, returns the maximum values within axis from predicted result.
y_pred = np.argmax(predictions_test, axis=1)

# Return the true data.
y_true = test_batches.classes

## Visualization
Calculate performance metrics (e.g., confusion matrix, precision, recall, etc) and visualize them.

In [ ]:
# Function for plotting confusion matrix graph to how the model predicted.
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    # set the figure size.
    plt.figure(figsize=(8,8)) #20,20
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    # set the title of figure.
    plt.title(title)
    # Plot the color bar for the figure
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    # normalize is set to False in this work
    if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
    else:
      print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, cm[i, j],
              horizontalalignment="center",
              color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    # Set y-axis of figure as True label
    plt.ylabel('True label')
    # Set x-axis of figure as Predicted label
    plt.xlabel('Predicted label')
    # Save the figure to specific path.
    plt.savefig('/home/ukawee_gmail_com/InceptionResNetV2/pic/testCM_150')

In [ ]:
# Plotting confusion matrix using confusion_matrix function from above
cm = confusion_matrix( y_true , y_pred)

# label the confusion matrix with the name of organs
organs = list(test_batches.class_indices.keys())
cm_plot_labels = organs

# Plot confusion matrix as output
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

# label for the classification table and print it as output. All value are round to 4 decimal places
target_names = test_batches.class_indices.keys()
print(classification_report(y_true, y_pred, target_names = target_names, digits=4))

In [ ]:
# Plotting table for specificity which doesn't provide by classification_report
# All value are round to 4 decimal places
from sklearn.metrics import precision_recall_fscore_support
res = []
for l in [0,1,2,3]:
    prec,recall,_,_ = precision_recall_fscore_support(np.array(y_true)==l,
                                                      np.array(y_pred)==l,
                                                      pos_label=True,average=None)
    res.append([l,round(recall[0],4),round((recall[1]),4)])

pd.DataFrame(res,columns = ['class','specificity','sensitivity (recall)'])